<a href="https://colab.research.google.com/github/kulottunga/first-contributions/blob/main/Helpmate_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
  #Install OpenAI and LlamaIndex
  !pip install -U -qq llama-index openai

In [22]:
#Install LangChain
!pip install -U -qq langchain langchain_community


In [23]:
# Step 1: Import the necessary libraries
import os
from llama_index.core import SimpleDirectoryReader  # Import adjusted based on your fix
import openai
from google.colab import drive



In [24]:
# Step 2: Mount your Google Drive and set the API key
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Set the API key
filepath = "/content/drive/MyDrive/HelpMate Project/"
with open(filepath + "OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [26]:
# Check if the path is correct
import os
filepath = "/content/drive/MyDrive/HelpMate Project/"
print(os.listdir(filepath))


['OpenAI_API_Key.txt', 'Policy_Doc']


In [30]:
# Step 3: Data Loading
data_folder_path = "/content/drive/MyDrive/HelpMate Project/Policy_Doc"
loader = SimpleDirectoryReader(data_folder_path)
documents = loader.load_data()

# Check if data loaded correctly
print(f"Number of documents loaded: {len(documents)}")

# Access the content of the first document using its text attribute
print(f"Sample document content: {documents[0].text[:500]}")  # Print the first 500 characters of the text


Number of documents loaded: 217
Sample document content:  
             Part A 
<<Date>> 
<<Policyholder’s Name>>  
<<Policyholder’s Address>> 
<<Policyholder’s Contact Number>>  
 
Dear <<Policyholder’s Name>>,  
 
Sub: Your Policy no. <<  >> 
We are glad to inform you that your proposal has been accepted and the HDFC Life Easy Health (“Policy”) 
being this document, has been issued. We have made every effort to design your Policy in a simple format. We 
have highlighted items of importance so that you may recognize them easily. 
 
Policy document:  


In [33]:
!pip install --upgrade llama-index


In [35]:
import llama_index
print(dir(llama_index))


['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'core', 'llms', 'readers']


In [36]:
import llama_index.core
print(dir(llama_index.core))


['BaseCallbackHandler', 'BasePromptTemplate', 'Callable', 'ChatPromptTemplate', 'ComposableGraph', 'Document', 'DocumentSummaryIndex', 'GPTDocumentSummaryIndex', 'GPTKeywordTableIndex', 'GPTListIndex', 'GPTRAKEKeywordTableIndex', 'GPTSimpleKeywordTableIndex', 'GPTTreeIndex', 'GPTVectorStoreIndex', 'IndexStructType', 'KeywordTableIndex', 'KnowledgeGraphIndex', 'ListIndex', 'MockEmbedding', 'NullHandler', 'Optional', 'Prompt', 'PromptHelper', 'PromptTemplate', 'PropertyGraphIndex', 'QueryBundle', 'RAKEKeywordTableIndex', 'Response', 'SQLContextBuilder', 'SQLDatabase', 'SQLDocumentContextBuilder', 'SelectorPromptTemplate', 'ServiceContext', 'Settings', 'SimpleDirectoryReader', 'SimpleKeywordTableIndex', 'StorageContext', 'SummaryIndex', 'TreeIndex', 'VectorStoreIndex', '__all__', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'async_utils', 'base', 'bridge', 'callbacks', 'chat_engine',

In [46]:
import os
from llama_index.core import VectorStoreIndex, StorageContext

# Create the directory if it doesn't exist
persist_dir = "/content/drive/MyDrive/HelpMate Project/Policy_Doc"
os.makedirs(persist_dir, exist_ok=True)



In [48]:
import os
from llama_index.core import VectorStoreIndex, StorageContext

# Define the directory for saving the index
persist_dir = "/content/drive/MyDrive/HelpMate Project/policy_index"

# Create the directory if it doesn't exist
os.makedirs(persist_dir, exist_ok=True)

# Create the index from documents
index = VectorStoreIndex.from_documents(documents)

# Persist the index directly to the specified directory
index.storage_context.persist(persist_dir)

print("Index created and saved successfully.")


Index created and saved successfully.


In [49]:
print("Directory contents before saving:", os.listdir(persist_dir))


Directory contents before saving: ['docstore.json', 'index_store.json', 'graph_store.json', 'default__vector_store.json', 'image__vector_store.json']


In [51]:
import os
from llama_index.core import VectorStoreIndex, StorageContext

# Define the directory for saving the index
persist_dir = "/content/drive/MyDrive/HelpMate Project/policy_index"

# Create the directory if it doesn't exist
os.makedirs(persist_dir, exist_ok=True)

# Create a storage context
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

# Create the index with the storage context
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# Persist the index using the storage context
storage_context.persist()

print("Index created and saved successfully.")


Index created and saved successfully.


In [54]:
# Step 5: Query the Index using as_query_engine()
if hasattr(index, 'as_query_engine'):
    query_engine = index.as_query_engine()
    response = query_engine.query("What is covered under the insurance policy?")
    print("Response:", response)
else:
    print("Query engine method not found. Check the documentation for updates.")


Response: The insurance policy covers the sum assured payable upon death or diagnosis of any one of the 29 covered critical illnesses during the coverage term. It also includes services rendered by the scheme member, voluntary participation for eligible persons, and certain medical treatments and conditions as specified in the policy document.


In [55]:
def response_pipeline(question):
    # Ensure the query engine is ready
    if not hasattr(index, 'as_query_engine'):
        raise AttributeError("Query engine not initialized.")

    # Create the query engine from the index
    query_engine = index.as_query_engine()

    # Query the engine with the user's question
    response = query_engine.query(question)

    # Format the response
    formatted_response = str(response)  # Convert response to a readable string if necessary
    return formatted_response

# Example usage of the response pipeline
user_question = "What are the benefits covered in the policy?"
response = response_pipeline(user_question)
print("Response:", response)


Response: The benefits covered in the policy include benefits on death, accidental death, accelerated critical illness option, and diagnosis of critical illnesses. The benefits payable vary depending on the specific event, such as death of the insured, diagnosis of covered critical illnesses, or undergoing specified surgeries.


In [56]:
def testing_pipeline(test_questions):
    # Ensure that the response pipeline works for multiple questions
    results = {}
    for i, question in enumerate(test_questions):
        try:
            response = response_pipeline(question)
            results[f"Question {i+1}"] = {
                "Question": question,
                "Response": response
            }
        except Exception as e:
            results[f"Question {i+1}"] = {
                "Question": question,
                "Error": str(e)
            }
    return results

# Define a set of test questions
test_questions = [
    "What are the benefits covered in the policy?",
    "How can I file a claim?",
    "What is the procedure for renewing my insurance?",
    "Does this policy cover pre-existing conditions?",
    "What is the deductible amount?"
]

# Run the testing pipeline
test_results = testing_pipeline(test_questions)

# Display the results
for key, value in test_results.items():
    print(f"{key}:\nQuestion: {value['Question']}\nResponse: {value.get('Response', 'Error: ' + value.get('Error', ''))}\n")


Question 1:
Question: What are the benefits covered in the policy?
Response: The benefits covered in the policy include:
- Sum Assured payable on death of the life assured
- Additional death benefit for accidental death
- Sum Assured payable on diagnosis of covered critical illnesses
- Surgical procedures covered with a percentage payout of the Annual Sum Assured based on the specific surgery undergone

Question 2:
Question: How can I file a claim?
Response: To file a claim, you need to submit the following documents in original:
- Duly filled and signed claim form
- Copy of Policy document (self attested copy)
- Claimant’s residence and identity proof (For all claims greater than Rs. 1 lakh)
- Cancelled personalized cheque or copy of first page of passbook in case of non-personalized cheque
- Discharge Summary (self attested copy)
- Final Hospital Bill (self attested copy)
- Medical records (self attested copies) including consultation notes, laboratory reports, X-Ray and MRI films
- 

Building a custom Promt Template

In [57]:
from llama_index.core import PromptTemplate

# Step 7: Define a custom prompt template
custom_prompt_template = PromptTemplate(
    template="You are an expert in insurance policies. Answer the following question in a detailed and informative manner:\n\nQuestion: {question}\n\nResponse:"
)

# Function to create a response with a custom prompt
def response_with_custom_prompt(question):
    # Ensure the query engine is initialized
    if not hasattr(index, 'as_query_engine'):
        raise AttributeError("Query engine not initialized.")

    # Create the query engine with the custom prompt template
    query_engine = index.as_query_engine(prompt_template=custom_prompt_template)

    # Query the engine
    response = query_engine.query(question)

    # Format the response
    formatted_response = str(response)
    return formatted_response

# Example usage
user_question = "What does this insurance policy cover?"
response = response_with_custom_prompt(user_question)
print("Response:", response)


Response: This insurance policy covers benefits related to death, accidental death, and diagnosis of critical illnesses for Scheme Members. The benefits include the payment of the Sum Assured in case of death, additional benefits for accidental death, and the Sum Assured upon diagnosis of any of the 29 covered critical illnesses during the Coverage Term.
